# Premature optimization is the root of all evil 
### - Donald Knuth

We look at a student solution that was able to solve the reverse problem rapidly

In [ ]:
#I included this function here because it is going to be used for Problems 4 and 5.
def is_palindrome(word):
    "Is the word a palindrome?"
    
    if len(word) == 0 or len(word) == 1:
        return True
    cLowerCaseBegin = word[0].lower()    #change the first character to lowercase
    cLowerCaseEnd = word[-1].lower()     #change the last character to lowercase
    if cLowerCaseBegin == cLowerCaseEnd: #compare the first and last characters
        return is_palindrome(word[1:-1]) #call again the function but remove the first and last characters
    return False

### What is going on here?

```python
    cLowerCaseBegin = word[0].lower() 
```

This is an example of Hungarian Notation - the 'c' specifies that this is a character.  

```python
    bReverseFound = False   #This variable indicates that a reversal string has been found.
```

You can read more about Hungarian Notaton here:

    https://en.wikipedia.org/wiki/Hungarian_notation
    
### Suggestions:

1) A long comment is difficult to read: I'd suggest

```python
    bReverseFound = False   # Have we seen the reverse?
```

2) The space after the '#' greatly increased legibility

### Left align the Happy Path

**Before**

```python
    if cLowerCaseBegin == cLowerCaseEnd: #compare the first and last characters
        return is_palindrome(word[1:-1]) #call again the function but remove the first and last characters
    return False
```

If the comments don't fit, you must aquit!  Space, caps, and compression

```python
    if cLowerCaseBegin == cLowerCaseEnd: # Compare first, last
        return is_palindrome(word[1:-1]) # Check remainder
    return False
```

I find this easier to read:


```python
    if cLowerCaseBegin != cLowerCaseEnd: # compare first, last  
        return False
    
    return is_palindrome(word[1:-1])  # check remainder
```

The "Happy Path" is now left aligned, rather than tucked in the if statement

In production code, there are typically dozens of checks for degenerate conditions.  Here is a simple example with three

**Before**

```python
    if len(lst) > 0:
        if word.isapla():
            if i >= 0:
                 # Do the real work
            else:
                <error condition for i>
                return None
        else:
            <error condition for word>
            return None
    else:
        <Error condition for list>
        return None
```

**After**
```python
    if len(lst) == 0:
        <Error condition for list>
        return None
    
    if not word.isapla():
        <error condition for word>
        return None
    
    if i < 0:
        <error condition for i>
        return None
            
    # Do the real work
```

### Rewrite is_palindrome

In [ ]:
#I included this function here because it is going to be used for Problems 4 and 5.
def is_palindrome(word):
    "Is the word a palindrome?"
    
    if len(word) == 0 or len(word) == 1:
        return True
    cLowerCaseBegin = word[0].lower()    # first character
    cLowerCaseEnd = word[-1].lower()     # last character 
    
    if cLowerCaseBegin != cLowerCaseEnd: # compare first, last 
        return False    # jdp
    
    return is_palindrome(word[1:-1])     # check remainder

### Binary Search - the key to the speedup

In [ ]:
#I included this function because it will be used in the succeeding problem.

def binary_search(list_of_words: list, word: str) -> int:
    """Takes a list of numbers and a target value
       Returns the position of the target in the list"""

    # Set boundaries
    low = 0
    high = len(list_of_words) - 1
    
    # Check subarray list_of_numbers from low to high
    while (low <= high) and not(low == high):
        mid = (low + high) // 2
        if word == list_of_words[mid]:
            # Found it!  Return index
            return mid
        elif word < list_of_words[mid]:
            high = mid 
        else:
            low = mid + 1

    # Not found 
    return -1

In [ ]:
### See line 10 above - What's up?
### See line 16 above - bug discussed in class

## We need a dataset to test Binary Search

### Routine to read the words

In [ ]:
def read_words(filename):
    "Return a list with the a representative of each pair of reversed element, such as abut and tuba"
    
    try:
        lst_to_traverse = []          #Initialize the list to traverse
        
        #Do some housekeeping by removing palindromes
        with open(filename, 'r') as words:
            for line in words:           
                line = line.strip()         #remove the end of line character 
                if not is_palindrome(line):  #remove any palindrome and any duplicate
                    lst_to_traverse.append(line.lower()) #add the word in the list 
    
    except FileNotFoundError:       # Send an error if the file is not found.
        print(f"Error ",filename," is not found. Please check file.")
        return []
        
    print("Number of words in list ",len(lst_to_traverse))  
    return lst_to_traverse

### Test of Binary Search

In [ ]:
def test_binary_search():
    lst = read_words('../words.txt')
    print(len(lst))
    for word in lst:
        if binary_search(lst, word) == -1:
            print(f'Could not find {word}')
        
test_binary_search()

### Where is zymurgy in the file words.txt?

In [ ]:
# A Unix command to look at end of the file
! tail ../words.txt

I see
```python
    zymogene
    zymogenes
    zymogens
    zymologies
    zymology
    zymoses
    zymosis
    zymotic
    zymurgies
    zymurgy
```

## We could not find the last element in the list

This didn't bite us in this problem, but is a real bug

### Two bugs

```python
    while (low <= high) and not(low == high):
```

Why phrase that this way?  Why not say

```python
    while (low < high):
```

This is a matter of software archeology.  This is from the buggy 
Binary Search I presented in lecutre.  My version read

```python
    while (low <= high):
```

but had an infinite loop.  The 'fix' means that we won't have an infinite loop, but it also means we won't find the last item.

There is no reason to quit if low == high - we will get mid == low == high, and check it.  

**Before**

```python
        mid = (low + high) // 2
        if word == list_of_words[mid]:
            # Found it!  Return index
            return mid
        elif word < list_of_words[mid]:
            high = mid 
        else:
            low = mid + 1
```

Notice the asymmetry between there lines

```python
            high = mid 
        else:
            low = mid + 1
```

The point of Binary Search is to split a subarray into three parts:
    
mid - the part we will check

front - the part before mid

read - the part after mid


```python
    ^           ^          ^
    |           |          |
   low         mid        high

```

After checking, we want to update to eliminate mid from further search

```python
    ^           ^          ^
    |           |          |
   low         mid        high

    ^          ^
    |          |
   low        high

```

Make a small change

**After**

```python
        mid = (low + high) // 2
        if word == list_of_words[mid]:
            # Found it!  Return index
            return mid
        elif word < list_of_words[mid]:
            high = mid - 1        # jdp - reduce subarray
        else:
            low = mid + 1
```

### Bottom line: there is a bug
### it spends a bit more time, and misses one cases
### But it was harmless in this problem

## Fix bugs in Binary Search
### And add two counters to track work done

In [ ]:
# Two counters to see how hard we worked
counter = 0    # How many times do we look at an item?
calls   = 0    # How many times do we call the function?

def binary_search(list_of_words: list, word: str) -> int:
    """Takes a list of numbers and a target value
       Returns the position of the target in the list"""
    global counter      # How many elements did we check?
    global calls        # How many times did we call this routine?
    calls = calls + 1
   
    # Set boundaries
    low = 0
    high = len(list_of_words) - 1
    
    # Check subarray list_of_numbers from low to high
    while (low <= high):              # jdp
        counter = counter + 1         # jdp
        mid = (low + high) // 2
        if word == list_of_words[mid]:
            # Found it!  Return index
            return mid
        elif word < list_of_words[mid]:
            high = mid - 1            # jdp
        else:
            low = mid + 1

    # Not found 
    return -1

def clear_counters():
    "Clear the counters"
    global counter
    global calls  
    
    counter = 0
    calls = 0

### Call Test again

In [ ]:
test_binary_search()

### Student's Find Reversals

In fact, he combined reading the file and looking for reverses

Read File and find reversals

This violates the "noun verb" rule for functions

Read File

Find Reversals


In [ ]:
def find_reversals_in_file(filename):
    "Return a list with the a representative of each pair of reversed element, such as abut and tuba"
    
    lst_to_traverse = read_words(filename)
    return_lst = []               #Initialize the list to return
 
    #Only check for reversal if there are two elements of the list
    while len(lst_to_traverse) > 1:
        word = lst_to_traverse[0]         #The first element is the one to be checked in the list.
        sWord_in_Reverse = word[::-1]     #Reverse the word to be searched.

        bEndOfList = False                #This variable indicates the end of the list has been reached.
        bReverseFound = False             #This variable indicates that a reversal string has been found.

        #If the word is already in the return list, remove it from the list.
        #If the word in reverse is also in the return list, remove it from the 
        #list too. Removing the word makes the list smaller.
        if word in return_lst or sWord_in_Reverse in return_lst:
            bWordReturned = True
            lst_to_traverse.pop(0)
        else:
            bWordReturned = False

        #If the word in reverse is found, then include it in the return list.
        #If it is not found and the end has been reached, exit out of the WHILE loop. 
        #Binary searched is used here to optimize searching for the word in reverse.
        while not bReverseFound and not bEndOfList:  
            if binary_search(lst_to_traverse,sWord_in_Reverse) != -1:
                bReverseFound = True
                return_lst.append(word.lower())
            else:
                bEndOfList = True
                
        #Removing the word makes the list smaller.
        if len(lst_to_traverse) > 0:
            lst_to_traverse.pop(0)
        
        #Removing the word in reverse makes the list smaller.
        if bReverseFound:
            lst_to_traverse.remove(sWord_in_Reverse)
                            
    return return_lst

In [ ]:
def test_find_reversals():
    !date
    clear_counters()
    lst = find_reversals_in_file('../words.txt')
    print(f'Length of results: {len(lst)}')
    print(f'Steps: {counter} Calls: {calls} Steps per call {counter/calls}')
    clear_counters()       # Belt and suspenders
    !date

test_find_reversals()

### What I see

```python
Sun Oct 18 13:34:18 MDT 2020
Number of words in list  113718
Length of results: 397
Steps: 1721826 Calls: 113320 Steps per call 15.194369925873632
Sun Oct 18 13:34:22 MDT 2020
```

### Overhead

Let's focus on what we need for a while loop: initialization, updates, check

```python
    lst_to_traverse = read_words(filename)
 
    #Only check for reversal if there are two elements of the list
    while len(lst_to_traverse) > 1:
        ...
        if word in return_lst or sWord_in_Reverse in return_lst:
            ...
            lst_to_traverse.pop(0)
                
        if len(lst_to_traverse) > 0:
            lst_to_traverse.pop(0)
        
        if bReverseFound:
            lst_to_traverse.remove(sWord_in_Reverse)
```

### If we don't pop() someplace, the list wont get any shorter.

### We suggest you use for loops, as there is less chance of error

### I will argue that there is little lost in efficiency

### What is going on here?

```python
    while not bReverseFound and not bEndOfList:  
        if binary_search(lst_to_traverse,sWord_in_Reverse) != -1:
            bReverseFound = True
            return_lst.append(word.lower())
        else:
            bEndOfList = True
```

I have a loop: but either bReverseFound or bEndOfList will be set true
    
That is, I only run through the loop once.  

I can remove the loop, and I can delete bEndOfList    

### What is going on here?

**Before**

```python
    while len(lst_to_traverse) > 1:
        word = lst_to_traverse[0]         #The first element is the one to be checked in the list.
 
        if word in return_lst or sWord_in_Reverse in return_lst:
            lst_to_traverse.pop(0)

        if len(lst_to_traverse) > 0:
            lst_to_traverse.pop(0)
```
### *We are processing the first word.  When wouldn't we pop it?*

### *Why would you have two pop()s and run the risk of missing a word?*

**After**
```python
    while len(lst_to_traverse) > 1:
        word = lst_to_traverse.pop(0) 
```


### What is going on here?

```python
    if word in return_lst or sWord_in_Reverse in return_lst:
        bWordReturned = True
        lst_to_traverse.pop(0)
    else:
        bWordReturned = False
```

Well, don't process if it has already been handled!

```python
    if not (word in return_lst or sWord_in_Reverse in return_lst):
        if binary_search(lst_to_traverse,sWord_in_Reverse) != -1:
```

### What is going on here?

***Before**

```python
        while not bReverseFound and not bEndOfList:  
            if binary_search(lst_to_traverse,sWord_in_Reverse) != -1:
                bReverseFound = True
        
        #Removing the word in reverse makes the list smaller.
        if bReverseFound:
            lst_to_traverse.remove(sWord_in_Reverse)                        
```

Don't set a flag - remove it now!

**After**

```python
        while not bReverseFound and not bEndOfList:  
            if binary_search(lst_to_traverse,sWord_in_Reverse) != -1:
                lst_to_traverse.remove(sWord_in_Reverse)                    
```

### Why throw away information?

We have found the position of the reverse: rather than asking the general
list remove() method to find it all over again, would be faster to call pop()


**Before**
```python
    if binary_search(lst_to_traverse,sWord_in_Reverse) != -1:
        ...

        lst_to_traverse.remove(sWord_in_Reverse)
```

#### *remove() doesn't know where the word is: its needs the search whole list for it*

**After**
```python
    pos = binary_search(lst_to_traverse,sWord_in_Reverse)
        ...
        lst_to_traverse.pop(pos)
```


This happens so rarely in oour problem that we won't see any change in the time.

## Rewrite

In [ ]:
  def find_reversals_in_file(filename):
    "Return a list with the a representative of each pair of reversed element, such as abut and tuba"
    
    lst_to_traverse = read_words(filename)
    return_lst = []               # Initialize return value
 
    # Check reversal if there are at least two elements
    while len(lst_to_traverse) > 1:
        word = lst_to_traverse.pop(0)   # Check first element of list
        sWord_in_Reverse = word[::-1]   # Find its reverse

        if not (word in return_lst or sWord_in_Reverse in return_lst):
            pos = binary_search(lst_to_traverse,sWord_in_Reverse)
            if  pos != -1:
                return_lst.append(word.lower())
                lst_to_traverse.pop(pos)
                            
    return return_lst

In [ ]:
test_find_reversals()

### What I see
```python
Sun Oct 18 14:02:08 MDT 2020
Number of words in list  113718
Length of results: 397
Steps: 1721776 Calls: 113320 Steps per call 15.193928697493822
Sun Oct 18 14:02:12 MDT 2020
```

#### Just as fast, and much easier to read and to maintain.

#### Preserves all the improvements of the original

In [ ]:
lst[:10]

## My Version: Rewrite routine to read words

### Skipping palindromes makes original less general.

There aren't enough plaindromes to make a perceptible difference

### Rewrite the error message using the power of f-string formating

**Before**
```python
    print(f"Error ",filename," is not found. Please check file.")
```

**After**
```python
    print(f"Error: {filename} was not found.") #    jdp - f-string
```

In [ ]:
def read_words(filename):
    "Return a list with contents of file"
    
    try:
        result = []
        
        with open(filename, 'r') as words:
            for line in words:           
                line = line.strip()
                result.append(line.lower())
    
        return result              # Move inside clause
    
    except FileNotFoundError:       # print message if file not found - jdp
        print(f"Error: {filename} was not found.") # jdp - f-string
        return []

## Search itself: version 1

The first version uses a for loop and the Binary Search

No attempt to remove elements: Binary Search is the only speedup

### Avoid indexing when you can

**Before**
```python
    for i in range(len(lst)):
        word = lst[i]
```

**After**
```python
    for word in lst:
```

### Conditions

```python
        if rev != word and rev not in result and word not in result:

```

#### Not a palidrome

```python
        if rev != word ...

```

#### Haven't seen rev


```python
        if ... rev not in result ...

```

#### Haven't seen word


```python
        if ... word not in result:

```


In [ ]:
# Enter your function  here
def find_reversals_in_file(filename):
    "Return a list with the a representative of each pair of reversed element"
    
    lst = read_words(filename) 
    
    result = []
    
    # For each word in the list
    for word in lst:
        
        # Find the reverse
        rev  = word[::-1]
        
        # If it isn't a palindrome, and we haven't seen it before
        if rev != word and rev not in result and word not in result:
            
            # Look for the reverse in the list
            if binary_search(lst, rev) != -1:
                
                # Found it: add word to our results
                result.append(word)
    return result

In [ ]:
test_find_reversals()

### What I see

```python
Sun Oct 18 14:06:53 MDT 2020
Length of results: 397
Steps: 1904573 Calls: 113321 Steps per call 16.806884866882573
Sun Oct 18 14:06:55 MDT 2020
```

### The secret sauce is the Binary Search

### The while loop casued undue complications

### Premature optimization is the root of all evil -- Donald Knuth

https://wiki.c2.com/?PrematureOptimization
    

In Donald Knuth's paper "Structured Programming With Go To Statements", 
he wrote: "Programmers waste enormous amounts of time thinking about, or worrying about, the speed of noncritical parts of their programs, and these attempts at efficiency actually have a strong negative impact when debugging and maintenance are considered. We should forget about small efficiencies, say about 97% of the time: premature optimization is the root of all evil. Yet we should not pass up our opportunities in that critical 3%."


### Small change: we only want to save half the words: 'abut', but not 'tuba'

There is a quick check: the words are sorted, so test if word < rev

**Before**
```python
    if rev != word and rev not in result and word not in result:
```

**After**
```python
    if word < rev:
```

We don't see the first word a second time, and this is a quck check that we don't have second word

#### Note that this also filters palindromes

### Version 2: This is a simple check that saves us half the lookups

In [ ]:
# Enter your function  here
def find_reversals_in_file(filename):
    "Return a list with the a representative of each pair of reversed element"
    
    lst = read_words(filename) 
    
    result = []
    
    # For each word in the list
    for word in lst:
        
        # Find the reverse
        rev  = word[::-1]
        
        # Representative will be smaller
        if word < rev:
            
            # Look for the reverse in the list
            if binary_search(lst, rev) != -1:
                
                # Found it: add word to our results
                result.append(word)
    return result

In [ ]:
test_find_reversals()

### What I see

```python
Sun Oct 18 14:18:22 MDT 2020
Length of results: 397
Steps: 1169218 Calls: 69357 Steps per call 16.857966751733784
Sun Oct 18 14:18:22 MDT 2020
```

### The code is short enoough to fit in a List Comprehension

In [ ]:
# Enter your function  here
def find_reversals_in_file(filename):
    "Return a list with the a representative of each pair of reversed element"
    
    lst = read_words(filename) 
    
    return [word for word in lst if word < word[::-1] 
            and binary_search(lst, word[::-1]) != -1]

In [ ]:
test_find_reversals()

### What I see

```python
Sun Oct 18 14:18:33 MDT 2020
Length of results: 397
Steps: 1169218 Calls: 69357 Steps per call 16.857966751733784
Sun Oct 18 14:18:34 MDT 2020
```

In [ ]:
lst[:10]

### KISS, and for loops are simpler than while loops.  

https://en.wikipedia.org/wiki/KISS_principle

The for loop was simpler, and no slower.  In fact, Version 1 was two times faster.  

A simple improvement made Version 2 four times faster.  

Here are a couple of articles explaning why for loops and List Comprehsnions are faster than while loops.

https://www.blog.duomly.com/loops-in-python-comparison-and-performance/

https://stackoverflow.com/questions/869229/why-is-looping-over-range-in-python-faster-than-using-a-while-loop

But that is not the argument we are making here.  

We want you to use for loops, as you are more likely to have a bug with a while loop than with a for loop.  

There is a place for While Loops - our Binary Search is a good example.  But KISS.

### Make it run, make it right, make it fast